In [ ]:
%%capture
import sys

# Añade el directorio principal al path de búsqueda para importar módulos desde esa ubicación
sys.path.insert(0, "..")

# Desactivar los warnings para evitar mensajes innecesarios durante la ejecución
import warnings

# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

from likelihood.models.deep import (
    AutoClassifier,
)  # Modelos de deep learning personalizados
from likelihood.tools import OneHotEncoder, get_metrics, apply_lora  # Herramientas auxiliares

import tensorflow as tf

In [2]:
# Cargar el dataset de cáncer de mama desde sklearn
df = datasets.load_breast_cancer()

# Convertir los datos a un DataFrame de pandas para facilitar la manipulación
df_cancer = pd.DataFrame(data=df.data, columns=df.feature_names)
df_cancer["target"] = df.target  # Añadir la columna de etiquetas 'target'

# OneHotEncoder convierte las etiquetas a formato one-hot encoding
y_encoder = OneHotEncoder()
y = y_encoder.encode(df_cancer["target"].to_list())  # Codificar las etiquetas de la clase (target)
X = df_cancer.drop(
    columns="target"
).to_numpy()  # Extraer las características (sin la columna 'target')
X = np.asarray(X).astype(np.float32)  # Convertir X a tipo float32 para la entrada del modelo
y = np.asarray(y).astype(np.float32)  # Convertir y a tipo float32

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Example usage
input_shape = (X.shape[1],)
num_classes = y.shape[1]

# Define the AutoClassifier model
model = AutoClassifier(
    input_shape_parm=input_shape[-1],
    num_classes=num_classes,
    units=17,
    activation="selu",
    l2_reg=0.0,
    num_layers=2,
    lora_mode=True,
)

# Compilación del modelo: optimizador, función de pérdida y métricas
model.compile(
    optimizer="adam",  # Optimizador Adam
    loss=tf.keras.losses.CategoricalCrossentropy(),  # Función de pérdida para clasificación multiclase
    metrics=[
        tf.keras.metrics.F1Score(threshold=0.5)
    ],  # Métrica F1 (threshold = 0.5 para predicciones)
)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=False)

# Hacer predicciones sobre el conjunto de entrenamiento
pred = model.predict(X)

# Convertir las predicciones a las etiquetas predichas (máxima probabilidad)
pred_label = np.argmax(pred, axis=1)

# Añadir las predicciones al DataFrame original para su análisis
df = df_cancer.copy()
y_labels = df.drop(columns="target").columns.to_list()
df_cancer["prediction"] = pred_label  # Columna de las etiquetas predichas
df_cancer["label_0"] = pred[:, 0]  # Probabilidad de la clase 0
df_cancer["label_1"] = pred[:, 1]  # Probabilidad de la clase 1

# Calcular y mostrar las métricas del modelo comparando las etiquetas reales y las predicciones
get_metrics(df_cancer, "target", "prediction", verbose=True)

# Guardar el modelo entrenado en el disco en formato TensorFlow
model.save("lora_model", save_format="tf")

Input shape: (None, 38)
Dense weights shape: 38x17
LoRA weights shape: A(38, 4), B(4, 17)
18/18 [==============================] - 0s 1ms/step
Accuracy: 91.74%
Precision: 88.94%
Recall: 99.16%
F1-Score: 93.77
Cohen's Kappa: 0.8161


In [4]:
model.classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LoRA_0 (LoRALayer)          (None, 17)                220       
                                                                 
 activation (Activation)     (None, 17)                0         
                                                                 
 dense_4 (Dense)             (None, 2)                 36        
                                                                 
Total params: 256 (1.00 KB)
Trainable params: 256 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Cargar el modelo guardado desde el disco
model = tf.keras.models.load_model("lora_model")
model.summary()
# Hacer predicciones nuevamente con el modelo cargado
pred = model.predict(X)

# Obtener las etiquetas predichas para las nuevas predicciones
pred_label = np.argmax(pred, axis=1)

# Añadir las nuevas predicciones al DataFrame original
df["prediction"] = pred_label

# Calcular y mostrar las métricas nuevamente con el modelo cargado
get_metrics(df, "target", "prediction", verbose=True)

AssertionError: Found 4 Python objects that were not bound to checkpointed values, likely due to changes in the Python program. Showing 4 of 4 unmatched objects: [<tf.Variable 'kernel:0' shape=(38, 17) dtype=float32, numpy=
array([[-0.08682367, -0.3138136 , -0.20465228, -0.00373584, -0.32972506,
         0.02088168,  0.1655002 ,  0.3237375 , -0.23878354,  0.22563583,
        -0.0238317 , -0.03691277, -0.23013854, -0.00163841, -0.07690227,
        -0.27483836, -0.16635971],
       [ 0.32840705, -0.00694564,  0.14094016,  0.2901131 ,  0.13030165,
         0.12820882, -0.0227012 ,  0.23289216,  0.2233954 , -0.17634791,
         0.10094994, -0.0710341 ,  0.00509053, -0.23132785, -0.23133186,
        -0.3034119 ,  0.18487024],
       [ 0.2136882 , -0.06205299,  0.20492852, -0.014164  ,  0.08798984,
         0.30495155,  0.15833545, -0.32297817, -0.16988239, -0.0037345 ,
         0.23332423, -0.02173758, -0.21040073, -0.2632298 ,  0.23551631,
        -0.09602039,  0.05802312],
       [ 0.238